In [2]:
pip install pgmpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 654 kB/s 


In [3]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
heartdisease_model=BayesianModel([('age','heartdisease'),('cp','heartdisease'),('heartdisease','restecg'),('heartdisease','chol')])
print('Bayesian network models are :')
print('\t',heartdisease_model.nodes())
print('Bayesian edges are:')
print('\t',heartdisease_model.edges())

Bayesian network models are :
	 ['age', 'heartdisease', 'cp', 'restecg', 'chol']
Bayesian edges are:
	 [('age', 'heartdisease'), ('heartdisease', 'restecg'), ('heartdisease', 'chol'), ('cp', 'heartdisease')]


/usr/local/lib/python3.7/dist-packages/pgmpy/models/BayesianModel.py:10: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  FutureWarning,


In [5]:
cpd_age = TabularCPD(variable='age', variable_card=2,
                      values=[[0.4], [0.6]])
#Age=0 (Young person below 30) Age =1 (Older person above 30) 
cpd_cp = TabularCPD(variable='cp', variable_card=2,
                       values=[[0.1], [0.9]])
#chestpain=0 (no chest pain), chestpain=1 (has chest pain)
cpd_heartdisease = TabularCPD(variable='heartdisease', variable_card=2,
                        values=[[0.99, 0.1, 0.8, 0.02],
                                [0.01, 0.9, 0.2, 0.98]],
                        evidence=['age', 'cp'],
                        evidence_card=[2, 2])
cpd_restecg = TabularCPD(variable='restecg', variable_card=2,
                      values=[[0.9, 0.2], [0.1, 0.8]],
                      evidence=['heartdisease'], evidence_card=[2])
#RestEcg=0 (Normal ECG) RestEcg=1 (Ubnormal ECG)
cpd_chol = TabularCPD(variable='chol', variable_card=2,
                      values=[[0.65, 0.3], [0.35, 0.7]],
                      evidence=['heartdisease'], evidence_card=[2])
#chol=0 (Normal Cholestrol) chol=1 (High Cholestrol)

In [6]:
# Associating the parameters with the model structure.
heartdisease_model.add_cpds(cpd_age, cpd_cp, cpd_heartdisease, cpd_restecg, cpd_chol)

# Checking if the cpds are valid for the model.
heartdisease_model.check_model()

True

In [7]:
heartdisease_infer=VariableElimination(heartdisease_model)

In [8]:
print('All local independecies are as follows')
heartdisease_model.get_independencies()
print('Displaying CPDs')
print(heartdisease_model.get_cpds('age'))
print(heartdisease_model.get_cpds('cp'))
print(heartdisease_model.get_cpds('heartdisease'))
print(heartdisease_model.get_cpds('restecg'))
print(heartdisease_model.get_cpds('chol'))

All local independecies are as follows
Displaying CPDs
+--------+-----+
| age(0) | 0.4 |
+--------+-----+
| age(1) | 0.6 |
+--------+-----+
+-------+-----+
| cp(0) | 0.1 |
+-------+-----+
| cp(1) | 0.9 |
+-------+-----+
+-----------------+--------+--------+--------+--------+
| age             | age(0) | age(0) | age(1) | age(1) |
+-----------------+--------+--------+--------+--------+
| cp              | cp(0)  | cp(1)  | cp(0)  | cp(1)  |
+-----------------+--------+--------+--------+--------+
| heartdisease(0) | 0.99   | 0.1    | 0.8    | 0.02   |
+-----------------+--------+--------+--------+--------+
| heartdisease(1) | 0.01   | 0.9    | 0.2    | 0.98   |
+-----------------+--------+--------+--------+--------+
+--------------+-----------------+-----------------+
| heartdisease | heartdisease(0) | heartdisease(1) |
+--------------+-----------------+-----------------+
| restecg(0)   | 0.9             | 0.2             |
+--------------+-----------------+-----------------+
| restecg(1

In [9]:
print('\n Probablity of heartdisease given chest pain')
q=heartdisease_infer.query(variables=['heartdisease'],evidence={'cp':1})
print(q)


 Probablity of heartdisease given chest pain


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.0520 |
+-----------------+---------------------+
| heartdisease(1) |              0.9480 |
+-----------------+---------------------+
